In [1]:
import numpy as np 
import os
import open3d as o3d
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import time
from scipy.spatial import cKDTree
from functools import reduce

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def mesh_to_point_cloud(mesh_path, num_points=1000):
    mesh = read_mesh(mesh_path)
    pcd = mesh.sample_points_uniformly(number_of_points=num_points)
    return pcd


def read_mesh(filename:str):
    #pcd = o3d.io.read_triangle_mesh(filename, True)
    pcd = o3d.io.read_triangle_model(filename, True)
    return pcd

def read_point_cloud(filename:str):
    pcd = o3d.io.read_point_cloud(filename)
    return pcd


def render(geometries):
    o3d.visualization.draw_geometries(geometries)


# side effect: will set renderer_pc's extrinsic camera
def get_view(renderer_pc, intrinsic, model, center, eye, up=[0,1,0], extrinsic=None, img_width=400, img_height=500):


    # grey = o3d.visualization.rendering.MaterialRecord()
    # grey.base_color = [0.7, 0.7, 0.7, 1.0]
    # grey.shader = "defaultLit"
    
    material = o3d.visualization.rendering.MaterialRecord()
    # texture = np.asarray(model.colors).copy()
    # texture = texture.astype(np.float32)
    # texture = o3d.geometry.Image(texture)
    # material.albedo_img = texture
    material.aspect_ratio = 1.0
    material.base_color = [0.7, 0.7, 0.7, 1.0]
    material.shader = "defaultUnlit"



    # renderer_pc.scene.scene.set_sun_light([-1, -1, -1], [1.0, 1.0, 1.0], 100000)
    renderer_pc.scene.scene.enable_sun_light(False)
    
    renderer_pc.scene.set_lighting(renderer_pc.scene.LightingProfile.HARD_SHADOWS, (0, 0, 0))
    # renderer_pc.scene.set_lighting(renderer_pc.scene.LightingProfile.NO_SHADOWS, (0, 0, 0))
    # renderer_pc.scene.scene.enable_sun_light(True)

    renderer_pc.scene.add_geometry("model", model, material)
    
    # # just for reference 
    # axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=([0., 0., 0.]))
    # renderer_pc.scene.add_geometry("axis", axis, material)

    
    # for triangle meshes
    # renderer_pc.scene.add_model("model", model)

    # Optionally set the camera field of view (to zoom in a bit)
    vertical_field_of_view = 30  # between 5 and 90 degrees
    aspect_ratio = img_width / img_height  # azimuth over elevation
    near_plane = 0.05
    far_plane = 50.0
    fov_type = o3d.visualization.rendering.Camera.FovType.Vertical
    # renderer_pc.scene.camera.set_projection(vertical_field_of_view, aspect_ratio, near_plane, far_plane, fov_type)
    
    renderer_pc.scene.camera.set_projection(intrinsic.intrinsic_matrix, near_plane, far_plane, img_width, img_height)

    # Look at the origin from the front (along the -Z direction, into the screen), with Y as Up.
    # camera orientation
    # extrinsic = lookat(np.asarray(center), np.asarray(eye), np.asarray(up))
    renderer_pc.scene.camera.look_at(center, eye, up)
    if extrinsic is not None:
        renderer_pc.setup_camera(intrinsic, extrinsic)
    # renderer_pc.setup_camera(intrinsic, extrinsic)

    depth_image = np.asarray(renderer_pc.render_to_depth_image(True) )
    image = np.asarray(renderer_pc.render_to_image())


    # plt.imshow(depth_image)
    renderer_pc.scene.remove_geometry("model")
    # renderer_pc.scene.remove_geometry("axis")
    return (image, depth_image, renderer_pc.scene.camera.get_view_matrix())

def recenter_model(model):
    mean_cov = model.compute_mean_and_covariance()
    mean = mean_cov[0]
    covariance_matrix = mean_cov[1]

    eigen_values, eigen_vectors = np.linalg.eigh(covariance_matrix)

    # 特征向量与特征值关联，并按特征值排序（特征值最大的对应主要的方向）
    eig_index = np.argsort(eigen_values)[::-1]
    principal_axes = eigen_vectors[:, eig_index]
    # principal_axes = principal_axes.transpose()

    # the first axis of pca should be y axis for a human
    principal_axes[:,[0,1]] = principal_axes[:,[1,0]]

    if principal_axes[:, 0] [0]< 0:
        principal_axes[:,0]*= -1
    if principal_axes[:,1][1] < 0:
        principal_axes[:,1]*= -1
    if principal_axes[:,2][2] < 0:
        principal_axes[:,2]*= -1
    # print(principal_axes)

    # new_z_axis = np.array([principal_axes[:,2][0], 0, principal_axes[:,2][1]])

    # z_axis = np.array([0, 0, 1])

    # rotation_axis = np.cross(z_axis, new_z_axis)
    # # print(rotation_axis)

    # # 计算旋转角度（dot product和arccos）
    # cos_theta = np.dot(z_axis, new_z_axis) / (np.linalg.norm(z_axis) * np.linalg.norm(new_z_axis))
    # angle = np.arccos(cos_theta)

    # # 计算旋转矩阵（Rodrigues' rotation formula）
    # K = np.array([
    #     [0, -rotation_axis[2], rotation_axis[1]],
    #     [rotation_axis[2], 0, -rotation_axis[0]],
    #     [-rotation_axis[1], rotation_axis[0], 0]
    # ])
    # identity_matrix = np.eye(3)
    # rotation_matrix = identity_matrix + np.sin(-angle) * K + (1 - np.cos(-angle)) * (K @ K)

    # model.rotate(rotation_matrix, center=mean)
    model.rotate(principal_axes.transpose(), center=mean)

    # mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.5, origin=mean)
    # mesh_frame.rotate(principal_axes, center=mean)
    # coorinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0., 0., 0.])
    # o3d.visualization.draw_geometries([model, mesh_frame,coorinate ] )
    
def lookat(center, eye, up):
    f = (eye - center)
    f /= np.linalg.norm(f)
    
    r = np.cross(up, f)
    r /= np.linalg.norm(r)
    
    u = np.cross(f, r)
    u /= np.linalg.norm(u)

    view_matrix = np.eye(4)
    view_matrix[:3, :3] = np.column_stack((r, u, -f))
    view_matrix[:3, 3] = -np.dot(np.column_stack((r, u, -f)), eye)
    
    return view_matrix


def flip(matrix):
    result = np.copy(matrix)
    result[1:3,:] = -1 * result[1:3,:]
    return result


def rgbd_to_pointcloud(rgbd, intrinsic, extrinsic_camera):
    # extrinsic_camera = np.array(extrinsic_camera)
    # extrinsic_camera = np.linalg.inv(extrinsic_camera)
    pcd =  o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, flip(extrinsic_camera))
    # Flip it, otherwise the pointcloud will be upside down
    #pcd.transform([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    return pcd

def rgbd_to_pointcloud2(rgbd, intrinsic, extrinsic_path):
    extrinsic  = np.load(extrinsic_path).astype(np.float32)
    pcd =  o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, flip(extrinsic))
    return pcd
def downsample_rescale(pcd, voxel_size=0.00001, scale=10000):
    downsampled_pc = pcd.voxel_down_sample(voxel_size)
    # use mean as the center for now
    a1 = np.asarray(downsampled_pc.points)
    scaled_downsampled_pc = downsampled_pc.scale(scale, a1.mean(0))
    return scaled_downsampled_pc


def downsample(pcd, voxel_size=0.00001):
    downsampled_pc = pcd.voxel_down_sample(voxel_size)
    return downsampled_pc


def read_segmentation(file_name, depth):
    # for read npy files
    rgb = np.load(file_name).astype(np.uint8)
    rgb = o3d.geometry.Image(rgb)
    ## for read image files
    #rgb = o3d.io.read_image(file_name)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgb, o3d.geometry.Image(depth), convert_rgb_to_intensity=False, depth_scale=1)
    return rgbd_image

def read_segmentation2(view_path, depth_path):
    # for read npy files
    rgb = np.load(view_path).astype(np.uint8)
    rgb = o3d.geometry.Image(rgb)
    depth = np.load(depth_path).astype(np.float32)
    ## for read image files
    #rgb = o3d.io.read_image(file_name)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgb, o3d.geometry.Image(depth), convert_rgb_to_intensity=False, depth_scale=1)
    return rgbd_image

def read_logits(view_path, depth_path):
    # for read npy files
    rgb = np.load(view_path).astype(np.float32)
    rgb = o3d.geometry.Image(rgb)
    depth = np.load(depth_path).astype(np.float32)
    ## for read image files
    #rgb = o3d.io.read_image(file_name)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgb, o3d.geometry.Image(depth), convert_rgb_to_intensity=False, depth_scale=1)
    return rgbd_image



def project(model_path, centers, eyes, renderer_pc, intrinsic, img_width, img_height):
    model = read_point_cloud(model_path)
    #IDEA recenter and rescale the model 
    points = np.asarray(model.points)
    points 
    points -= points.mean(0)
    model.transform(np.array([[1,0,0,0], 
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]]))
    recenter_model(model)

    Image = []
    Depth = []
    Extrinsic = []
    for i in range(len(eyes)):
        image, depth, extrinsic = get_view(renderer_pc, intrinsic, model, center=centers[i], eye=eyes[i], up=[0,1,0], img_width=img_width, img_height=img_height)
        Image.append(image)
        Depth.append(depth)
        Extrinsic.append(extrinsic)
    return Image, Depth, Extrinsic

def reproject(Masks, Depth, intrinsic, Extrinsic):
    # Pcd = []
    Pcd = o3d.geometry.PointCloud()
    for i in range(len(Masks)):
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
            o3d.geometry.Image(Masks[i].astype(np.float32)), o3d.geometry.Image(Depth[i]), convert_rgb_to_intensity=False, depth_scale=1)
        pcd = rgbd_to_pointcloud(rgbd_image, intrinsic, Extrinsic[i])
        Pcd = pcd + Pcd
    return Pcd

def save_pcd(pcd, file_path):
    o3d.io.write_point_cloud(file_path, pcd)

def knn_for_all_conflict_points(Pcd):
    K = 100

    Pcd_points = np.asarray(Pcd.points)
    Pcd_colors = np.asarray(Pcd.colors)
    # knn for one point
    def knn(location):
        points = Pcd_points
        distances = np.linalg.norm(points - location, ord=2, axis=1)
        inds = np.argsort(distances)[:K]
        labels = Pcd_colors[inds]
        unique_labels, counts = np.unique(labels, axis=0, return_counts=True)
        return unique_labels[np.argsort(counts)[-1]]
    
    uniques, counts = np.unique(Pcd_points, axis=0, return_counts=True)
    duplicate_points = uniques[counts > 1]
    all_inds = np.arange(0, len(Pcd_points))
    print("Find {} duplicate points".format(len(duplicate_points)))
    for point in duplicate_points:
        inds = all_inds[np.all(Pcd_points == point, axis=1)]
        colors = Pcd_colors[inds]
        if len(np.unique(colors, axis=0)) == 1:
            continue
        else:
            label = knn(point)
        Pcd_colors[inds][:] = label
    
    pcd_ret = o3d.geometry.PointCloud()
    pcd_ret.points = o3d.utility.Vector3dVector(Pcd_points)
    pcd_ret.colors = o3d.utility.Vector3dVector(Pcd_colors)

    return pcd_ret
    
def list_all_files(directory):
    all_files = []
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            all_files.append(os.path.join(dirpath, filename))
    return all_files

def knn_with_confidence(K, location, tree, colors, logits, unique_labels, useMean=True):
        _, inds = tree.query(location, k=K)
        labels = colors[inds]
        confidentials = logits[inds]
        results = np.zeros((unique_labels.shape[0],))
        
        for i, unique_label in enumerate(unique_labels):
        # Find all rows in labels that match the current unique_label
            matches = np.all(labels == unique_label, axis=1)
            if np.any(matches):  # Check if there are any matches
                # sum -> mean
                if useMean:
                    results[i] = np.mean(confidentials[matches])
                else:
                    results[i] = np.sum(confidentials[matches])/len(inds)
            else:
                results[i] = 0  # or some other value indicating no matches

        if np.all(unique_labels[np.argmax(results)] == [0,0,0]) and results[np.argsort(results)[-2]] > 0.1:
            result = unique_labels[np.argsort(results)[-2]]
        else:
            result = unique_labels[np.argmax(results)]
        # result = unique_labels[np.argmax(results)]
        # unique_labels, counts = np.unique(labels, axis=0, return_counts=True)
        # result = unique_labels[np.argsort(counts)[-1]]
        # #IDEA assumption for human models: no backgrounds
        # if (np.array_equal(result, background_label)):
        #     # if there is other choice
        #     if len(unique_labels) > 1:
        #         result = unique_labels[np.argsort(counts)[-2]]
        #     # choose the cloest point 
        #     else:
        #         distances, inds = tree.query(location, k=4000)
        #         non_background_distances = []
        #         for i in range(4000):
        #             if not np.array_equal(colors[inds[i]], background_label):
        #                 non_background_distances.append((distances[i], colors[inds[i]]))
        #         if non_background_distances:
        #             result = min(non_background_distances, key=lambda x: x[0])[1]
        return result
    
def knn_with_confidence_within_radius(radius, location, tree, colors, logits, unique_labels, useMean = True):
    # _, inds = tree.query(location, k=K)
    inds = tree.query_ball_point(location, radius)
    # [k, inds, _]=tree.search_radius_vector_3d(location, 0.1)
    labels = colors[inds]
    confidentials = logits[inds]
    results = np.zeros((unique_labels.shape[0],))
    
    # for label, confidential in zip(labels, confidentials):
    #     # Find the index of the label in unique_labels
    #     idx = np.where(np.all(unique_labels == label, axis=1))[0]
    #     if idx.size > 0:
    #         results[idx[0]] += confidential[0]
    for i, unique_label in enumerate(unique_labels):
    # Find all rows in labels that match the current unique_label
        matches = np.all(labels == unique_label, axis=1)
        if np.any(matches):  # Check if there are any matches
            # Sum the corresponding confidential values
            if useMean:
                results[i] = np.mean(confidentials[matches])
            else:
                results[i] = np.sum(confidentials[matches])/len(inds)
        else:
            results[i] = 0  # or some other value indicating no matches
    if np.all(unique_labels[np.argmax(results)] == [0,0,0]) and results[np.argsort(results)[-2]] > 0.1:
            result = unique_labels[np.argsort(results)[-2]]
    else:
        result = unique_labels[np.argmax(results)]
    # result = unique_labels[np.argmax(results)]
        # print(results[i])
    # print(results)
    # if unique_labels[np.argmax(results)] == [0,0,0]:
        
        


    return result
    
def knn(K, location, tree, colors):
    _, inds = tree.query(location, k=K)
    labels = colors[inds]

    unique_labels, counts = np.unique(labels, axis=0, return_counts=True)
    result = unique_labels[np.argsort(counts)[-1]]

    return result

def knn_with_confidence_within_radius_weighted_mean_count(radius, location, tree, colors, logits, unique_labels, weight=0.7):
    # _, inds = tree.query(location, k=K)
    inds = tree.query_ball_point(location, radius)
    total = len(inds)
    # [k, inds, _]=tree.search_radius_vector_3d(location, 0.1)
    labels = colors[inds]
    confidentials = logits[inds]
    results = np.zeros((unique_labels.shape[0],))
    
    # for label, confidential in zip(labels, confidentials):
    #     # Find the index of the label in unique_labels
    #     idx = np.where(np.all(unique_labels == label, axis=1))[0]
    #     if idx.size > 0:
    #         results[idx[0]] += confidential[0]
    for i, unique_label in enumerate(unique_labels):
    # Find all rows in labels that match the current unique_label
        matches = np.all(labels == unique_label, axis=1)
        if np.any(matches):  # Check if there are any matches
            # Sum the corresponding confidential values
            
                results[i] = weight * np.mean(confidentials[matches]) + (1-weight) * confidentials[matches].shape[0]/total
            
        else:
            results[i] = 0  # or some other value indicating no matches
    if np.all(unique_labels[np.argmax(results)] == [0,0,0]) and results[np.argsort(results)[-2]] > 0.1:
            result = unique_labels[np.argsort(results)[-2]]
    else:
        result = unique_labels[np.argmax(results)]
    return result

def knn_with_confidence_weighted_mean_count(K, location, tree, colors, logits, unique_labels, weight=0.7):
        _, inds = tree.query(location, k=K)
        
        labels = colors[inds]
        confidentials = logits[inds]
        results = np.zeros((unique_labels.shape[0],))
        
        for i, unique_label in enumerate(unique_labels):
        # Find all rows in labels that match the current unique_label
            matches = np.all(labels == unique_label, axis=1)
            if np.any(matches):  # Check if there are any matches
                # sum -> mean
                 results[i] = weight * np.mean(confidentials[matches]) + (1-weight) * confidentials[matches].shape[0]/len(inds)
            else:
                results[i] = 0  # or some other value indicating no matches

        if np.all(unique_labels[np.argmax(results)] == [0,0,0]) and results[np.argsort(results)[-2]] > 0.1:
            result = unique_labels[np.argsort(results)[-2]]
        else:
            result = unique_labels[np.argmax(results)]

        return result



In [3]:
# files = list_all_files()
models = [ '/home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply',
          '/home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply']
intrinsic =  o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.Kinect2ColorCameraDefault)
class AnyObject:
    pass
args = AnyObject() 
args.mask_input_path = '/home/demoy/3dProjection/cdcl_output'
args.depth_input_path = '/home/demoy/3dProjection/depths'
args.extrinsic_input_path = '/home/demoy/3dProjection/extrinsics'
args.confidence_input_path = '/home/demoy/3dProjection/segs'
args.num_views = 10
args.k = 100
args.radius = 0.05
args.smooth = 100
args.output_path = '/home/demoy/3dProjection/confidence_search'

#part_ids = [ 0,  2,  4,  5,  6,  8, 13, 14, 16, 17, 18, 19, 20, 21, 24]
# the order is: (left right flipped)
# background, head, torso, left upper arm ,right upper arm, left forearm, right forearm,
#  left hand, right hand, left thigh, right thigh, left shank, right shank, left foot, right foot
part_ids = [0, 13, 2, 5, 8, 19, 20, 4, 24, 18, 6, 21, 16, 14, 17]

r_chan = np.array([0, 127, 255, 255, 255, 127, 255, 127, 0, 0, 0, 0, 127, 255, 255])
g_chan = np.array( [0, 127, 0, 127, 255, 0, 0, 127, 255, 0, 255, 127, 255, 127, 255])
b_chan = np.array( [0, 127, 0, 0, 0, 255, 255, 0, 255, 255, 0, 255, 127, 127, 127])

In [4]:
# model size
for model in models:
    pcd = read_point_cloud(model)
    obox = pcd.get_axis_aligned_bounding_box()
    print(obox.max_bound - obox.min_bound)

[0.558004 1.999462 0.704354]
[0.88784  1.999873 0.942558]


In [7]:
# test radius size 
model_point_cloud = read_point_cloud(f'/home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply')
points = np.asarray(model_point_cloud.points)
points 
points -= points.mean(0)
model_point_cloud.transform(np.array([[1,0,0,0], 
                    [0,1,0,0],
                    [0,0,1,0],
                    [0,0,0,1]]))
recenter_model(model_point_cloud)

sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.07, resolution=30)
sphere.translate((-0.4,0.5,0))
render([model_point_cloud, o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0]), sphere])

In [ ]:
# read results
t = read_point_cloud(f'frame_01141.ply')
render([t])

In [8]:
def human_seg_combine_channel(human_seg_split_map):
    r_chan_seg = np.add.reduce(human_seg_split_map * np.array(r_chan), 2)
    g_chan_seg = np.add.reduce(human_seg_split_map * np.array(g_chan), 2)
    b_chan_seg = np.add.reduce(human_seg_split_map * np.array(b_chan), 2)
    return np.stack([b_chan_seg, g_chan_seg, r_chan_seg], axis=-1).astype(np.uint8)

def human_seg_combine_argmax(human_seg_argmax_map):
    onehot = np.stack([(human_seg_argmax_map == i).astype(np.uint8) for i in range(15)], axis=-1)
    return human_seg_combine_channel(onehot)

In [ ]:
ks= [500, 600, 700, 800, 900, 1000]
# ks=[500]
dis = [0.05, 0.06, 0.07, 0.08, 0.09]
history_time = []
for k in ks:
    
    for d in dis:
        args.radius = d
        print('set radius to ', d)
        
        for model in models:
            start_time = time.time()
            print("Processsing model {}".format(model))
            id = model.split(os.sep)[-2]
            model_point_cloud = read_point_cloud(model)
            points = np.asarray(model_point_cloud.points)
            points 
            points -= points.mean(0)
            model_point_cloud.transform(np.array([[1,0,0,0], 
                                [0,1,0,0],
                                [0,0,1,0],
                                [0,0,0,1]]))
            recenter_model(model_point_cloud)
            ############ 2D to 3D Reprojection #############
            
            pcds = []
            confidences = []
            for i in range(10):
                # idea: reconstruct a point cloud, whose color will be index to an array containing 
                # all pixel labels/confidence
                view_path = os.path.join(args.mask_input_path, f"{id}_image{i+1}_cdcl_segmask.npy")
                logit_path = os.path.join(args.confidence_input_path, f'{id}_image{i+1}_cdcl_seg.npy')
                depth_path = os.path.join(args.depth_input_path, f"{id}_depth{i+1}.npy")
                extrinsic_path = os.path.join(args.extrinsic_input_path, f"{id}_extrinsic{i+1}.npy")
                
                rgb = np.load(view_path).astype(np.uint8)
                depth = np.load(depth_path).astype(np.float32)
                extrinsic = np.load(extrinsic_path).astype(np.float32)
                # shape(H*W*15 for cdcl)
                confidence = np.load(logit_path).astype(np.float32)
                # extract height and width i.e number of pixels
                shape = rgb.shape[:2]
                # build the index array, for different views, just continue extend the index
                # read open3d api, the create_from_color_and_depth has some limitation on the 
                # conbination of data type and color channel, so have to make the index 4 byte data type
                # error: because of the precision of floating numbers, index are not unique
                index = np.arange(i*shape[0]*shape[1], (i+1)*shape[0]*shape[1]).astype(np.float32)
                # reshape to the shape as the depth map
                index = np.reshape(index, (shape))
                
                rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
                    o3d.geometry.Image(index), o3d.geometry.Image(depth), convert_rgb_to_intensity=False, depth_scale=1)
                
                pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, flip(extrinsic), False)
                
                
                pcds.append(pcd)
                confidences.append(confidence.reshape((shape[0]*shape[1],confidence.shape[-1])))
            # assert(len(pcds) == args.num_views)
            
            Pcd_merged = (reduce(lambda x, y: x + y, pcds))


            confidences = np.array(confidences)
            confidences = np.reshape(confidences,(len(pcds)*shape[0]*shape[1], confidence.shape[-1]))
            
            
            seg_points = np.asarray(Pcd_merged.points)
            tree = cKDTree(seg_points)

            index = np.asarray(Pcd_merged.colors)

            # each index should be unique
            assert(seg_points.shape[0] == np.unique(seg_points,axis=0).shape[0])
            assert(index.shape[0] == np.unique(index,axis=0).shape[0])

            points = np.asarray(model_point_cloud.points)
            
            labels = []
            for point in tqdm(points, desc="Assign Labels", unit="point") :
                _, inds = tree.query(point, k=500)
                
                assert(inds.shape[0] == np.unique(inds,axis=0).shape[0])
                # print(inds)
                # inds = tree.query_ball_point(location, radius)
                knn_index = index[inds,0].astype(int)
                # shape(K*15)
                knn_confidences = confidences[knn_index]
                
                # compute distance matrix
                distance = np.sqrt(np.sum((seg_points[inds]-point)**2,axis=-1))
                # print(distance)
                assert(distance.shape==inds.shape)
                
                # filter points that is too far
                knn_confidences_within_distance = knn_confidences[distance < 0.07]
                
                # todo: deal with empty knn_confidences_within_distance
                
                # merge and sort the result to get labels' mean confidence (1*15 for cdcl)
                labels_mean = np.mean(knn_confidences_within_distance,axis=0)
                
                seg_argmax = np.argmax(labels_mean, axis=-1)
                seg_max = np.max(labels_mean, axis=-1)
                
                # seg_max_thres = (seg_max > 0.1).astype(np.uint8)
                # seg_argmax *= seg_max_thres
                
                result = seg_argmax

                if seg_argmax==0:
                    if(labels_mean[np.argsort(labels_mean)[-2]]/seg_max > 0.5):
                        result = np.argsort(labels_mean)[-2]
                elif seg_max < 0.1:
                    result = 0
                labels.append( result)
            
            labels_stacked = np.array(labels).astype(np.uint8)
            
            # assign colors based on label index
            colors_values = np.vstack((b_chan[labels_stacked], g_chan[labels_stacked], r_chan[labels_stacked])).T
            pcd_result = o3d.geometry.PointCloud()
            pcd_result.points = o3d.utility.Vector3dVector(points)
            pcd_result.colors = o3d.utility.Vector3dVector(colors_values)
            render([pcd_result])
            

In [13]:
# current method!
ks= [1000,1200,1400, 1600, 1800, 2000]
# ks=[500]
dis = [0.1, 0.11]
history_time = []
for k in ks:
    args.k = k
    print(f'set k to {k}')
    for d in dis:
        args.radius = d
        print('set radius to ', d)
        
        for model in models:
            start_time = time.time()
            print("Processsing model {}".format(model))
            id = model.split(os.sep)[-2]
            model_point_cloud = read_point_cloud(model)
            points = np.asarray(model_point_cloud.points)
            points -= points.mean(0)
            model_point_cloud.transform(np.array([[1,0,0,0], 
                                [0,1,0,0],
                                [0,0,1,0],
                                [0,0,0,1]]))
            recenter_model(model_point_cloud)
            ############ 2D to 3D Reprojection #############
            
            pcds = []
            confidences = []
            for i in range(10):
                # idea: reconstruct a point cloud using all pixels no matter the depths are valid or not
                # from the source code, the point list is of the order of the image raw by raw. So the point list
                # of the point cloud can be mapped to pixel confidence without other data structure
                #
                view_path = os.path.join(args.mask_input_path, f"{id}_image{i+1}_cdcl_segmask.npy")
                logit_path = os.path.join(args.confidence_input_path, f'{id}_image{i+1}_cdcl_seg.npy')
                depth_path = os.path.join(args.depth_input_path, f"{id}_depth{i+1}.npy")
                extrinsic_path = os.path.join(args.extrinsic_input_path, f"{id}_extrinsic{i+1}.npy")
                
                rgb = np.load(view_path).astype(np.uint8)
                depth = np.load(depth_path).astype(np.float32)
                extrinsic = np.load(extrinsic_path).astype(np.float32)
                # shape(H*W*15 for cdcl)
                confidence = np.load(logit_path).astype(np.float32)
                # extract height and width i.e number of pixels
                shape = rgb.shape[:2]
                
                pcd = o3d.geometry.PointCloud.create_from_depth_image(o3d.geometry.Image(depth), intrinsic, flip(extrinsic), depth_scale=1, project_valid_depth_only=False)

                pcds.append(pcd)
                
                # # check for pcd gotten from depth image
                # seg_points = np.asarray(pcd.points)
                # # get index of points with valid depth value
                # valid_points_index = np.where(~np.any(np.isnan(seg_points), axis=1))[0]
                # valid_points = seg_points[valid_points_index]
                # pcd_result = o3d.geometry.PointCloud()
                # pcd_result.points = o3d.utility.Vector3dVector(valid_points)
                # render([pcd_result])
                
                confidences.append(confidence.reshape((shape[0]*shape[1],confidence.shape[-1])))
            
            Pcd_merged = (reduce(lambda x, y: x + y, pcds))

            
            confidences = np.array(confidences)
            # make all confidences from all views a 1-d array to have a one-to-one mapping to the point list in the point cloud
            confidences = np.reshape(confidences,(len(pcds)*shape[0]*shape[1], confidence.shape[-1]))
            
            
            seg_points = np.asarray(Pcd_merged.points)

            # get index of points with valid depth value
            valid_points_index = np.where(~np.any(np.isnan(seg_points), axis=1))[0]
            valid_points = seg_points[valid_points_index]
            # build tree with valid points
            tree = cKDTree(valid_points)

            points = np.asarray(model_point_cloud.points)
            
            labels = []
            for point in tqdm(points, desc="Assign Labels", unit="point") :
                _, inds = tree.query(point, k=args.k)
                
                # inds2 = tree.query_ball_point(point, args.radius)
                # intersection = np.intersect1d(inds, inds2)
                # if intersection.shape[0] == 0:
                #     result = 0
                #     labels.append( result)
                #     continue
                
                # get confidence using index in all points
                knn_index = valid_points_index[inds].astype(int)
                # shape(K*15)
                knn_confidences = confidences[knn_index]
                
                knn_confidences_within_distance = knn_confidences
                
                
                # compute distance matrix of valid points to the target point
                distance = np.sqrt(np.sum((valid_points[inds]-point)**2,axis=-1))
                # print(distance)
                assert(distance.shape==inds.shape)
                
                # filter points that is too far
                knn_confidences_within_distance = knn_confidences[distance <= args.radius]
                
                # deal with empty knn_confidences_within_distance
                if knn_confidences_within_distance.shape[0] == 0:
                    result = 0
                    labels.append( result)
                    continue
                # merge and sort the result to get labels' mean confidence (1*15 for cdcl)
                labels_mean = np.mean(knn_confidences_within_distance,axis=0)
                
                seg_argmax = np.argmax(labels_mean, axis=-1)
                seg_max = np.max(labels_mean, axis=-1)
                
                
                result = seg_argmax
                # background thresholding
                if seg_argmax==0:
                    if(labels_mean[np.argsort(labels_mean)[-2]]/seg_max > 0.5):
                        result = np.argsort(labels_mean)[-2]
                elif seg_max < 0.1:
                    result = 0
                labels.append( result)
            
            labels_stacked = np.array(labels).astype(np.uint8)
            
            # assign colors based on label index
            # open3d point cloud colors should be normalized
            colors_values = np.vstack((b_chan[labels_stacked], g_chan[labels_stacked], r_chan[labels_stacked])).T/255
            pcd_result = o3d.geometry.PointCloud()
            assert(colors_values.shape[0] == points.shape[0])
            pcd_result.points = o3d.utility.Vector3dVector(points)
            pcd_result.colors = o3d.utility.Vector3dVector(colors_values)
            # render([pcd_result])
            
            # smoothing
            points = np.asarray(pcd_result.points)
            seg_colors = np.asarray(pcd_result.colors)
            tree = cKDTree(points)
            colors = []
            for point in tqdm(points, desc="Processing", unit="point") :
                color = knn(args.smooth, point, tree, seg_colors)
                # break
                colors.append( color)
                
            colors_stacked = np.vstack(colors)

            pcd_result = o3d.geometry.PointCloud()
            pcd_result.points = o3d.utility.Vector3dVector(points)
            pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
            
            end_time = time.time()
            time_used = (int) (end_time - start_time)
            print('time used:', time_used)
            # render([pcd_result])
            path = os.path.join(args.output_path, 'all_seg')
            # o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{d}_result_{time_used}.ply'),pcd_result )
            o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{args.k}_{args.radius}_{args.smooth}_result_{time_used}.ply'),pcd_result )
            print(f"{id} result point cloud output to {os.path.join(path,f'{id}_cdcl_pcd_{args.k}_{args.radius}_{args.smooth}_result_{time_used}.ply')}")

                        

set k to 1000
set radius to  0.1
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply


KeyboardInterrupt: 

In [27]:
ks= [500,600,700,800,900,1000]
# ks=[500]
dis = [0.03, 0.04, 0.05, 0.06, 0.07]
history_time = []
for d in dis:
    args.radius = d
    print('set radius to ', d)
    
    for model in models:
        start_time = time.time()
        print("Processsing model {}".format(model))
        id = model.split(os.sep)[-2]
        model_point_cloud = read_point_cloud(model)
        points = np.asarray(model_point_cloud.points)
        points 
        points -= points.mean(0)
        model_point_cloud.transform(np.array([[1,0,0,0], 
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]]))
        recenter_model(model_point_cloud)
        ############ 2D to 3D Reprojection #############
        
        pcds = []
        logits = []
        for i in range(10):
            view_path = os.path.join(args.mask_input_path, f"{id}_image{i+1}_cdcl_segmask.npy")
            logit_path = os.path.join(args.mask_input_path, f'{id}_image{i+1}_cdcl_seg_output.npy')
            depth_path = os.path.join(args.depth_input_path, f"{id}_depth{i+1}.npy")
            extrinsic_path = os.path.join(args.extrinsic_input_path, f"{id}_extrinsic{i+1}.npy")
            rgbd_image_seg = read_segmentation2(view_path=view_path, depth_path=depth_path)
            logit = read_logits(view_path=logit_path, depth_path=depth_path)
            
            print(f"{id} get view from {id}_image{i+1}_cdcl_segmask.npy")
            pcd = rgbd_to_pointcloud2(rgbd_image_seg, intrinsic, extrinsic_path)   
            logit = rgbd_to_pointcloud2(logit, intrinsic, extrinsic_path)     
            pcds.append(pcd)
            logits.append(logit)
        assert(len(pcds) == args.num_views)

        Pcd_merged = (reduce(lambda x, y: x + y, pcds))
        path = os.path.join(args.output_path, 'raw')
        # o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{d}_result_{time_used}.ply'),pcd_result )
        o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_.ply'),Pcd_merged )
        render([Pcd_merged])
        logits_merged = reduce(lambda x, y: x + y, logits)
        seg_points = np.asarray(Pcd_merged.points)
        tree = cKDTree(seg_points)

        seg_colors = np.asarray(Pcd_merged.colors)
        seg_logits = np.asarray(logits_merged.colors)
        # print(np.max(seg_logits))
        unique_labels,_ = np.unique(seg_colors, axis=0, return_counts=True)
        # print(unique_labels)
        points = np.asarray(model_point_cloud.points)
        colors = []
        for point in tqdm(points, desc="Assign Labels", unit="point") :
            # color = knn_with_confidence_within_radius(d, point, tree, seg_colors, seg_logits, unique_labels)
            color = knn_with_confidence_within_radius(args.radius, point, tree, seg_colors, seg_logits, unique_labels)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)

        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        # smoothing
        points = np.asarray(pcd_result.points)
        seg_colors = np.asarray(pcd_result.colors)
        tree = cKDTree(points)
        colors = []
        for point in tqdm(points, desc="Processing", unit="point") :
            color = knn(args.smooth, point, tree, seg_colors)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)

        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        end_time = time.time()
        time_used = (int) (end_time - start_time)
        print('time used:', time_used)
        history_time.append(time_used)
        # render([pcd_result])
        path = os.path.join(args.output_path, 'radius+useMean+backgroundThreshold+smooth')
        # o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{d}_result_{time_used}.ply'),pcd_result )
        o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{args.radius}_{args.smooth}_result_{time_used}.ply'),pcd_result )
        # print(f"{id} result point cloud output to {args.output_path}")

    
    

set radius to  0.03
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply
0501 get view from 0501_image1_cdcl_segmask.npy
0501 get view from 0501_image2_cdcl_segmask.npy
0501 get view from 0501_image3_cdcl_segmask.npy
0501 get view from 0501_image4_cdcl_segmask.npy
0501 get view from 0501_image5_cdcl_segmask.npy
0501 get view from 0501_image6_cdcl_segmask.npy
0501 get view from 0501_image7_cdcl_segmask.npy
0501 get view from 0501_image8_cdcl_segmask.npy
0501 get view from 0501_image9_cdcl_segmask.npy
0501 get view from 0501_image10_cdcl_segmask.npy


Assign Labels:   0%|          | 3586/799964 [00:01<04:45, 2788.88point/s]


KeyboardInterrupt: 

In [6]:
ks= [500,600,700,800,900,1000]
# ks=[500]
dis = [0.03, 0.04, 0.05, 0.06, 0.07]
history_time = []
for d in dis:
    args.radius = d
    print('set radius to ', d)
    
    for model in models:
        start_time = time.time()
        print("Processsing model {}".format(model))
        id = model.split(os.sep)[-2]
        model_point_cloud = read_point_cloud(model)
        points = np.asarray(model_point_cloud.points)
        points 
        points -= points.mean(0)
        model_point_cloud.transform(np.array([[1,0,0,0], 
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]]))
        recenter_model(model_point_cloud)
        ############ 2D to 3D Reprojection #############
        
        pcds = []
        logits = []
        for i in range(10):
            view_path = os.path.join(args.mask_input_path, f"{id}_image{i+1}_cdcl_segmask.npy")
            logit_path = os.path.join(args.mask_input_path, f'{id}_image{i+1}_cdcl_seg_output.npy')
            depth_path = os.path.join(args.depth_input_path, f"{id}_depth{i+1}.npy")
            extrinsic_path = os.path.join(args.extrinsic_input_path, f"{id}_extrinsic{i+1}.npy")
            rgbd_image_seg = read_segmentation2(view_path=view_path, depth_path=depth_path)
            logit = read_logits(view_path=logit_path, depth_path=depth_path)
            
            print(f"{id} get view from {id}_image{i+1}_cdcl_segmask.npy")
            pcd = rgbd_to_pointcloud2(rgbd_image_seg, intrinsic, extrinsic_path)   
            logit = rgbd_to_pointcloud2(logit, intrinsic, extrinsic_path)     
            pcds.append(pcd)
            logits.append(logit)
        assert(len(pcds) == args.num_views)

        Pcd_merged = (reduce(lambda x, y: x + y, pcds))
        # render([Pcd_merged])
        logits_merged = reduce(lambda x, y: x + y, logits)
        seg_points = np.asarray(Pcd_merged.points)
        tree = cKDTree(seg_points)

        seg_colors = np.asarray(Pcd_merged.colors)
        seg_logits = np.asarray(logits_merged.colors)
        # print(np.max(seg_logits))
        unique_labels,_ = np.unique(seg_colors, axis=0, return_counts=True)
        # print(unique_labels)
        points = np.asarray(model_point_cloud.points)
        colors = []
        for point in tqdm(points, desc="Assign Labels", unit="point") :
            # color = knn_with_confidence_within_radius(d, point, tree, seg_colors, seg_logits, unique_labels)
            color = knn_with_confidence_within_radius(args.radius, point, tree, seg_colors, seg_logits, unique_labels, useMean=False)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)

        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        # smoothing
        points = np.asarray(pcd_result.points)
        seg_colors = np.asarray(pcd_result.colors)
        tree = cKDTree(points)
        colors = []
        for point in tqdm(points, desc="Processing", unit="point") :
            color = knn(args.smooth, point, tree, seg_colors)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)

        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        end_time = time.time()
        time_used = (int) (end_time - start_time)
        print('time used:', time_used)
        history_time.append(time_used)
        # render([pcd_result])
        path = os.path.join(args.output_path, 'radius+useSum+backgroundThreshold+smooth')
        # o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{d}_result_{time_used}.ply'),pcd_result )
        o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{args.radius}_{args.smooth}_result_{time_used}.ply'),pcd_result )

set radius to  0.03
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799951/799951 [01:11<00:00, 11253.50point/s]


time used: 387
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply
0501 get view from 0501_image1_cdcl_segmask.npy
0501 get view from 0501_image2_cdcl_segmask.npy
0501 get view from 0501_image3_cdcl_segmask.npy
0501 get view from 0501_image4_cdcl_segmask.npy
0501 get view from 0501_image5_cdcl_segmask.npy
0501 get view from 0501_image6_cdcl_segmask.npy
0501 get view from 0501_image7_cdcl_segmask.npy
0501 get view from 0501_image8_cdcl_segmask.npy
0501 get view from 0501_image9_cdcl_segmask.npy
0501 get view from 0501_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799964/799964 [01:11<00:00, 11265.21point/s]


time used: 398
set radius to  0.04
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


...

time used: 1165
set radius to  0.07
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799951/799951 [01:11<00:00, 11216.96point/s]


time used: 1484
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply
0501 get view from 0501_image1_cdcl_segmask.npy
0501 get view from 0501_image2_cdcl_segmask.npy
0501 get view from 0501_image3_cdcl_segmask.npy
0501 get view from 0501_image4_cdcl_segmask.npy
0501 get view from 0501_image5_cdcl_segmask.npy
0501 get view from 0501_image6_cdcl_segmask.npy
0501 get view from 0501_image7_cdcl_segmask.npy
0501 get view from 0501_image8_cdcl_segmask.npy
0501 get view from 0501_image9_cdcl_segmask.npy
0501 get view from 0501_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799964/799964 [01:10<00:00, 11386.36point/s]


time used: 1542


In [7]:
ks= [500,600,700,800,900,1000]
dis = [0.03, 0.04, 0.05, 0.06, 0.07]
history_time = []
for d in dis:
    args.radius = d
    print('set radius to ', d)
    
    for model in models:
        start_time = time.time()
        print("Processsing model {}".format(model))
        id = model.split(os.sep)[-2]
        model_point_cloud = read_point_cloud(model)
        points = np.asarray(model_point_cloud.points)
        points 
        points -= points.mean(0)
        model_point_cloud.transform(np.array([[1,0,0,0], 
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]]))
        recenter_model(model_point_cloud)
        ############ 2D to 3D Reprojection #############
        
        pcds = []
        logits = []
        for i in range(10):
            view_path = os.path.join(args.mask_input_path, f"{id}_image{i+1}_cdcl_segmask.npy")
            logit_path = os.path.join(args.mask_input_path, f'{id}_image{i+1}_cdcl_seg_output.npy')
            depth_path = os.path.join(args.depth_input_path, f"{id}_depth{i+1}.npy")
            extrinsic_path = os.path.join(args.extrinsic_input_path, f"{id}_extrinsic{i+1}.npy")
            rgbd_image_seg = read_segmentation2(view_path=view_path, depth_path=depth_path)
            logit = read_logits(view_path=logit_path, depth_path=depth_path)
            
            print(f"{id} get view from {id}_image{i+1}_cdcl_segmask.npy")
            pcd = rgbd_to_pointcloud2(rgbd_image_seg, intrinsic, extrinsic_path)   
            logit = rgbd_to_pointcloud2(logit, intrinsic, extrinsic_path)     
            pcds.append(pcd)
            logits.append(logit)
        assert(len(pcds) == args.num_views)

        Pcd_merged = (reduce(lambda x, y: x + y, pcds))
        # render([Pcd_merged])
        logits_merged = reduce(lambda x, y: x + y, logits)
        seg_points = np.asarray(Pcd_merged.points)
        tree = cKDTree(seg_points)

        seg_colors = np.asarray(Pcd_merged.colors)
        seg_logits = np.asarray(logits_merged.colors)
        # print(np.max(seg_logits))
        unique_labels,_ = np.unique(seg_colors, axis=0, return_counts=True)
        # print(unique_labels)
        points = np.asarray(model_point_cloud.points)
        colors = []
        for point in tqdm(points, desc="Assign Labels", unit="point") :
            # color = knn_with_confidence_within_radius(d, point, tree, seg_colors, seg_logits, unique_labels)
            color = knn_with_confidence_within_radius_weighted_mean_count(args.radius, point, tree, seg_colors, seg_logits, unique_labels)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)

        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        # smoothing
        points = np.asarray(pcd_result.points)
        seg_colors = np.asarray(pcd_result.colors)
        tree = cKDTree(points)
        colors = []
        for point in tqdm(points, desc="Processing", unit="point") :
            color = knn(args.smooth, point, tree, seg_colors)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)

        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        end_time = time.time()
        time_used = (int) (end_time - start_time)
        print('time used:', time_used)
        history_time.append(time_used)
        # render([pcd_result])
        path = os.path.join(args.output_path, 'radius+weighted+backgroundThreshold+smooth')
        # o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{d}_result_{time_used}.ply'),pcd_result )
        o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{args.radius}_{args.smooth}_result_{time_used}.ply'),pcd_result )

set radius to  0.03
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799951/799951 [01:10<00:00, 11286.51point/s]


time used: 406
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply
0501 get view from 0501_image1_cdcl_segmask.npy
0501 get view from 0501_image2_cdcl_segmask.npy
0501 get view from 0501_image3_cdcl_segmask.npy
0501 get view from 0501_image4_cdcl_segmask.npy
0501 get view from 0501_image5_cdcl_segmask.npy
0501 get view from 0501_image6_cdcl_segmask.npy
0501 get view from 0501_image7_cdcl_segmask.npy
0501 get view from 0501_image8_cdcl_segmask.npy
0501 get view from 0501_image9_cdcl_segmask.npy
0501 get view from 0501_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799964/799964 [01:11<00:00, 11256.71point/s]


time used: 419
set radius to  0.04
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


...

time used: 1231
set radius to  0.07
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799951/799951 [01:12<00:00, 11047.00point/s]


time used: 1585
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply
0501 get view from 0501_image1_cdcl_segmask.npy
0501 get view from 0501_image2_cdcl_segmask.npy
0501 get view from 0501_image3_cdcl_segmask.npy
0501 get view from 0501_image4_cdcl_segmask.npy
0501 get view from 0501_image5_cdcl_segmask.npy
0501 get view from 0501_image6_cdcl_segmask.npy
0501 get view from 0501_image7_cdcl_segmask.npy
0501 get view from 0501_image8_cdcl_segmask.npy
0501 get view from 0501_image9_cdcl_segmask.npy
0501 get view from 0501_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799964/799964 [01:17<00:00, 10265.73point/s]


time used: 1748


In [64]:
ks= [500, 800, 1000, 1200, 1500, 1700, 2000]
dis = [0.03, 0.04, 0.05, 0.06, 0.07]
history_time = []
for k in ks:
    args.k = k
    print('set k to ', k)
    
    for model in models:
        start_time = time.time()
        print("Processsing model {}".format(model))
        id = model.split(os.sep)[-2]
        model_point_cloud = read_point_cloud(model)
        points = np.asarray(model_point_cloud.points)
        points 
        points -= points.mean(0)
        model_point_cloud.transform(np.array([[1,0,0,0], 
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]]))
        recenter_model(model_point_cloud)
        ############ 2D to 3D Reprojection #############
        
        pcds = []
        logits = []
        for i in range(10):
            view_path = os.path.join(args.mask_input_path, f"{id}_image{i+1}_cdcl_segmask.npy")
            logit_path = os.path.join(args.mask_input_path, f'{id}_image{i+1}_cdcl_seg_output.npy')
            depth_path = os.path.join(args.depth_input_path, f"{id}_depth{i+1}.npy")
            extrinsic_path = os.path.join(args.extrinsic_input_path, f"{id}_extrinsic{i+1}.npy")
            rgbd_image_seg = read_segmentation2(view_path=view_path, depth_path=depth_path)
            logit = read_logits(view_path=logit_path, depth_path=depth_path)
            
            print(f"{id} get view from {id}_image{i+1}_cdcl_segmask.npy")
            pcd = rgbd_to_pointcloud2(rgbd_image_seg, intrinsic, extrinsic_path)   
            logit = rgbd_to_pointcloud2(logit, intrinsic, extrinsic_path)     
            pcds.append(pcd)
            logits.append(logit)
        assert(len(pcds) == args.num_views)

        Pcd_merged = (reduce(lambda x, y: x + y, pcds))
        # render([Pcd_merged])
        logits_merged = reduce(lambda x, y: x + y, logits)
        seg_points = np.asarray(Pcd_merged.points)
        tree = cKDTree(seg_points)

        seg_colors = np.asarray(Pcd_merged.colors)
        seg_logits = np.asarray(logits_merged.colors)
        # print(np.max(seg_logits))
        unique_labels,_ = np.unique(seg_colors, axis=0, return_counts=True)
        # print(unique_labels)
        points = np.asarray(model_point_cloud.points)
        colors = []
        for point in tqdm(points, desc="Assign Labels", unit="point") :
            # color = knn_with_confidence_within_radius(d, point, tree, seg_colors, seg_logits, unique_labels)
            color = knn_with_confidence(args.k, point, tree, seg_colors, seg_logits, unique_labels, useMean=True)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)

        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        # smoothing
        points = np.asarray(pcd_result.points)
        seg_colors = np.asarray(pcd_result.colors)
        tree = cKDTree(points)
        colors = []
        for point in tqdm(points, desc="Processing", unit="point") :
            color = knn(args.smooth, point, tree, seg_colors)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)

        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        end_time = time.time()
        time_used = (int) (end_time - start_time)
        print('time used:', time_used)
        history_time.append(time_used)
        # render([pcd_result])
        path = os.path.join(args.output_path, 'k+useMean+backgroundThreshold+smooth')
        # o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{d}_result_{time_used}.ply'),pcd_result )
        o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{args.k}_{args.smooth}_result_{time_used}.ply'),pcd_result )

set k to  500
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799951/799951 [01:11<00:00, 11225.62point/s]


time used: 313
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply
0501 get view from 0501_image1_cdcl_segmask.npy
0501 get view from 0501_image2_cdcl_segmask.npy
0501 get view from 0501_image3_cdcl_segmask.npy
0501 get view from 0501_image4_cdcl_segmask.npy
0501 get view from 0501_image5_cdcl_segmask.npy
0501 get view from 0501_image6_cdcl_segmask.npy
0501 get view from 0501_image7_cdcl_segmask.npy
0501 get view from 0501_image8_cdcl_segmask.npy
0501 get view from 0501_image9_cdcl_segmask.npy
0501 get view from 0501_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799964/799964 [01:09<00:00, 11437.57point/s]


time used: 318
set k to  800
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


...

time used: 635
set k to  2000
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799951/799951 [01:12<00:00, 10973.98point/s]


time used: 756
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply
0501 get view from 0501_image1_cdcl_segmask.npy
0501 get view from 0501_image2_cdcl_segmask.npy
0501 get view from 0501_image3_cdcl_segmask.npy
0501 get view from 0501_image4_cdcl_segmask.npy
0501 get view from 0501_image5_cdcl_segmask.npy
0501 get view from 0501_image6_cdcl_segmask.npy
0501 get view from 0501_image7_cdcl_segmask.npy
0501 get view from 0501_image8_cdcl_segmask.npy
0501 get view from 0501_image9_cdcl_segmask.npy
0501 get view from 0501_image10_cdcl_segmask.npy


Processing: 100%|██████████| 799964/799964 [01:11<00:00, 11155.36point/s]


time used: 763


In [63]:
ks= [2100, 2200, 2150]
dis = [0.03, 0.04, 0.05, 0.06, 0.07]

history_time = []
for k in ks:
    args.k = k
    print('set k to ', k)
    # args.smooth = 500
    for model in models:
        start_time = time.time()
        print("Processsing model {}".format(model))
        id = model.split(os.sep)[-2]
        model_point_cloud = read_point_cloud(model)
        points = np.asarray(model_point_cloud.points)
        points 
        points -= points.mean(0)
        model_point_cloud.transform(np.array([[1,0,0,0], 
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]]))
        recenter_model(model_point_cloud)
        ############ 2D to 3D Reprojection #############
        
        pcds = []
        logits = []
        for i in range(10):
            view_path = os.path.join(args.mask_input_path, f"{id}_image{i+1}_cdcl_segmask.npy")
            logit_path = os.path.join(args.mask_input_path, f'{id}_image{i+1}_cdcl_seg_output.npy')
            depth_path = os.path.join(args.depth_input_path, f"{id}_depth{i+1}.npy")
            extrinsic_path = os.path.join(args.extrinsic_input_path, f"{id}_extrinsic{i+1}.npy")
            rgbd_image_seg = read_segmentation2(view_path=view_path, depth_path=depth_path)
            logit = read_logits(view_path=logit_path, depth_path=depth_path)
            
            print(f"{id} get view from {id}_image{i+1}_cdcl_segmask.npy")
            pcd = rgbd_to_pointcloud2(rgbd_image_seg, intrinsic, extrinsic_path)   
            logit = rgbd_to_pointcloud2(logit, intrinsic, extrinsic_path)     
            pcds.append(pcd)
            logits.append(logit)
        assert(len(pcds) == args.num_views)

        Pcd_merged = (reduce(lambda x, y: x + y, pcds))
        # render([Pcd_merged])
        logits_merged = reduce(lambda x, y: x + y, logits)
        seg_points = np.asarray(Pcd_merged.points)
        tree = cKDTree(seg_points)

        seg_colors = np.asarray(Pcd_merged.colors)
        seg_logits = np.asarray(logits_merged.colors)
        # print(np.max(seg_logits))
        unique_labels,_ = np.unique(seg_colors, axis=0, return_counts=True)
        # print(unique_labels)
        points = np.asarray(model_point_cloud.points)
        colors = []
        for point in tqdm(points, desc="Assign Labels", unit="point") :
            # color = knn_with_confidence_within_radius(d, point, tree, seg_colors, seg_logits, unique_labels)
            # color = knn_with_confidence_weighted_mean_count(args.k, point, tree, seg_colors, seg_logits, unique_labels)
            color = knn_with_confidence(K=args.k, location=point, tree=tree, colors=seg_colors, logits=seg_logits, unique_labels=unique_labels, useMean=False)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)

        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        # smoothing
        # points = np.asarray(pcd_result.points)
        # seg_colors = np.asarray(pcd_result.colors)
        # tree = cKDTree(points)
        # colors = []
        # for point in tqdm(points, desc="Processing", unit="point") :
        #     color = knn(args.smooth, point, tree, seg_colors)
        #     # break
        #     colors.append( color)
            
        # colors_stacked = np.vstack(colors)

        # pcd_result = o3d.geometry.PointCloud()
        # pcd_result.points = o3d.utility.Vector3dVector(points)
        # pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        
        end_time = time.time()
        time_used = (int) (end_time - start_time)
        print('time used:', time_used)
        history_time.append(time_used)
        # render([pcd_result])
        path = os.path.join(args.output_path, 'k+useSum+backgroundThreshold+smooth')
        # o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{d}_result_{time_used}.ply'),pcd_result )
        o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{args.k}_0_result_{time_used}.ply'),pcd_result )

set k to  2100
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


Assign Labels: 100%|██████████| 799951/799951 [11:46<00:00, 1131.84point/s]


time used: 708
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply
0501 get view from 0501_image1_cdcl_segmask.npy
0501 get view from 0501_image2_cdcl_segmask.npy
0501 get view from 0501_image3_cdcl_segmask.npy
0501 get view from 0501_image4_cdcl_segmask.npy
0501 get view from 0501_image5_cdcl_segmask.npy
0501 get view from 0501_image6_cdcl_segmask.npy
0501 get view from 0501_image7_cdcl_segmask.npy
0501 get view from 0501_image8_cdcl_segmask.npy
0501 get view from 0501_image9_cdcl_segmask.npy
0501 get view from 0501_image10_cdcl_segmask.npy


Assign Labels: 100%|██████████| 799964/799964 [11:54<00:00, 1119.04point/s]


time used: 717
set k to  2200
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


...

time used: 735
set k to  2150
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply
0500 get view from 0500_image1_cdcl_segmask.npy
0500 get view from 0500_image2_cdcl_segmask.npy
0500 get view from 0500_image3_cdcl_segmask.npy
0500 get view from 0500_image4_cdcl_segmask.npy
0500 get view from 0500_image5_cdcl_segmask.npy
0500 get view from 0500_image6_cdcl_segmask.npy
0500 get view from 0500_image7_cdcl_segmask.npy
0500 get view from 0500_image8_cdcl_segmask.npy
0500 get view from 0500_image9_cdcl_segmask.npy
0500 get view from 0500_image10_cdcl_segmask.npy


Assign Labels: 100%|██████████| 799951/799951 [12:20<00:00, 1079.99point/s]


time used: 745
Processsing model /home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply
0501 get view from 0501_image1_cdcl_segmask.npy
0501 get view from 0501_image2_cdcl_segmask.npy
0501 get view from 0501_image3_cdcl_segmask.npy
0501 get view from 0501_image4_cdcl_segmask.npy
0501 get view from 0501_image5_cdcl_segmask.npy
0501 get view from 0501_image6_cdcl_segmask.npy
0501 get view from 0501_image7_cdcl_segmask.npy
0501 get view from 0501_image8_cdcl_segmask.npy
0501 get view from 0501_image9_cdcl_segmask.npy
0501 get view from 0501_image10_cdcl_segmask.npy


Assign Labels: 100%|██████████| 799964/799964 [12:22<00:00, 1077.86point/s]


time used: 746


In [13]:
# smooth = [100,200,300,400, 500]
# pointclouds = [f'/home/demoy/3dProjection/confidence_search/k+weighted+backgroundThreshold+smooth/0500_cdcl_pcd_2100_0_result_753.ply', 
#                f'/home/demoy/3dProjection/confidence_search/k+weighted+backgroundThreshold+smooth/0500_cdcl_pcd_2150_0_result_762.ply',
#                f'/home/demoy/3dProjection/confidence_search/k+weighted+backgroundThreshold+smooth/0500_cdcl_pcd_2200_0_result_776.ply',
#                f'/home/demoy/3dProjection/confidence_search/k+weighted+backgroundThreshold+smooth/0501_cdcl_pcd_2100_0_result_748.ply',
#                f'/home/demoy/3dProjection/confidence_search/k+weighted+backgroundThreshold+smooth/0501_cdcl_pcd_2150_0_result_768.ply',
#                f'/home/demoy/3dProjection/confidence_search/k+weighted+backgroundThreshold+smooth/0501_cdcl_pcd_2200_0_result_786.ply']

smooth = [400, 500]
pointclouds = [f'/home/demoy/3dProjection/confidence_search/k+useSum+backgroundThreshold+smooth/0501_cdcl_pcd_2200_0_result_735.ply']
for s in smooth:
    print('set smooth to ', s)
    for pd in pointclouds:
        
        filename = os.path.basename(pd)
        print(f'work on {filename}')
        filename = os.path.splitext(filename)[0]
        
        fileargs = filename.split('_')
        timeused = fileargs[-1]
        k = fileargs[3]
        id = fileargs[0]
        
        start_time = time.time()
        pcd_result = read_point_cloud(pd)
        points = np.asarray(pcd_result.points)
        seg_colors = np.asarray(pcd_result.colors)
        tree = cKDTree(points)
        colors = []
        for point in tqdm(points, desc="Processing", unit="point") :
            color = knn(s, point, tree, seg_colors)
            # break
            colors.append( color)
            
        colors_stacked = np.vstack(colors)
        end_time = time.time()
        
        total_time = end_time-start_time+float(timeused)
        total_time = int(total_time)
        pcd_result = o3d.geometry.PointCloud()
        pcd_result.points = o3d.utility.Vector3dVector(points)
        pcd_result.colors = o3d.utility.Vector3dVector(colors_stacked)
        path = os.path.join(args.output_path, 'k+weighted+backgroundThreshold+smooth')
            # o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{d}_result_{time_used}.ply'),pcd_result )
        o3d.io.write_point_cloud(os.path.join(path,f'{id}_cdcl_pcd_{k}_{s}_result_{total_time}.ply'),pcd_result )

set smooth to  400
work on 0501_cdcl_pcd_2200_0_result_735.ply


Processing: 100%|██████████| 799964/799964 [02:43<00:00, 4886.02point/s]


set smooth to  500
work on 0501_cdcl_pcd_2200_0_result_735.ply


Processing: 100%|██████████| 799964/799964 [03:09<00:00, 4212.05point/s]


In [18]:
t = read_point_cloud(f'/home/demoy/3dProjection/confidence_search/raw/0501_cdcl_pcd_.ply')

render([t])

In [103]:
k = 400
t = read_point_cloud(f'/home/demoy/3dProjection/confidence_search/useMean/0500_cdcl_pcd_500_result.ply')

render([t])